# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell, Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

    CondaPkg Found dependencies: /home/runner/.julia/packages/ASEconvert/CNQ1A/CondaPkg.toml
    CondaPkg Found dependencies: /home/runner/.julia/packages/PythonCall/1f5yE/CondaPkg.toml
    CondaPkg Dependencies already up to date


As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123


Python: None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298190694302                   -0.85    4.8         
  2   -8.300176904276       -2.70       -1.26    1.2    187ms
  3   -8.300430255409       -3.60       -1.89    2.8    155ms
  4   -8.300460849533       -4.51       -2.72    2.2    143ms
  5   -8.300464373508       -5.45       -3.24    2.6    168ms
  6   -8.300464510819       -6.86       -3.37    9.8    296ms
  7   -8.300464570680       -7.22       -3.51    1.2    154ms
  8   -8.300464607556       -7.43       -3.65    8.1    322ms
  9   -8.300464633954       -7.58       -3.86    1.5    150ms
 10   -8.300464639826       -8.23       -4.00    2.0    149ms
 11   -8.300464643495       -8.44       -4.30    4.5    247ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.64187239369                   -0.70    5.4         
  2   -16.64606254605       -2.38       -1.14    1.0    321ms
  3   -16.67919072391       -1.48       -1.88    2.9    408ms
  4   -16.67927745897       -4.06       -2.65    3.4    408ms
  5   -16.67928598583       -5.07       -3.00    5.0    525ms
  6   -16.67928619032       -6.69       -3.34    2.0    337ms
  7   -16.67928621621       -7.59       -3.86    1.9    323ms
  8   -16.67928622132       -8.29       -4.55    3.2    403ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32671383523                   -0.56    6.8         
  2   -33.33438552224       -2.12       -1.00    1.2    1.25s
  3   -33.33588641330       -2.82       -1.72    5.5    1.63s
  4   -33.33611957322       -3.63       -2.62    5.9    1.69s
  5   -33.33677554943       -3.18       -2.21    6.6    1.94s
  6   -33.33680949276       -4.47       -2.26    1.6    1.19s
  7   -33.33694287210       -3.87       -3.43    1.5    1.28s
  8   -33.33694338692       -6.29       -3.49    4.4    1.88s
  9   -33.33694369623       -6.51       -3.78    1.5    1.13s
 10   -33.33694378174       -7.07       -4.73    2.0    1.31s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298402478273                   -0.85    6.2         
  2   -8.300258750147       -2.73       -1.58    1.0   97.6ms
  3   -8.300433170786       -3.76       -2.56    2.2    146ms
  4   -8.300384933198   +   -4.32       -2.30    4.0    183ms
  5   -8.300463936304       -4.10       -3.37    1.1    112ms
  6   -8.300464559827       -6.21       -3.74    6.5    237ms
  7   -8.300464633787       -7.13       -4.18    1.1    117ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/CNQ1A/src/ASEconvert.jl:123
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32486968235                   -0.56    7.2         
  2   -33.29138657016   +   -1.48       -1.25    1.0    1.01s
  3   +5.742140842620   +    1.59       -0.27    6.9    2.42s
  4   -33.32931231820        1.59       -1.84    6.8    2.21s
  5   -33.17116854775   +   -0.80       -1.31    5.6    2.01s
  6   -32.41479438366   +   -0.12       -1.08    4.6    1.76s
  7   -33.30311072671       -0.05       -1.72    4.2    1.60s
  8   -33.33456441080       -1.50       -2.18    2.4    1.24s
  9   -33.33555486905       -3.00       -2.18    3.1    1.45s
 10   -33.33651656821       -3.02       -2.70    1.8    1.19s
 11   -33.33693854258       -3.37       -3.13    3.2    1.44s
 12   -33.33694147211       -5.53      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.